In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)                #결과 재현을 위해 랜덤 시드를 정해주자.as_integer_ratio

x_data=[[1,2], [2,3], [3,1], [4,3], [5,3], [6,2]]
y_data=[[0],[0],[0],[1],[1],[1]]

x_train=torch.FloatTensor(x_data)
y_train=torch.FloatTensor(y_data)

W=torch.zeros((2,1),requires_grad=True)
b=torch.zeros(1,requires_grad=True)

# 직접 sigmoid 함수를 만들어서 넣는 방법
# hypothesis=1/(1+torch.exp(x_train.matmul(W)+b))
###################################################
# 하지만 sigmoid 함수는 torch에서 제공을 해준다.
hypothesis=torch.sigmoid(x_train.matmul(W)+b)

# cost function을 직접 만드는 방법
# losses=-(y_train*torch.log(hypothesis)+(1-y_train)*torch.log(hypothesis))
# cost=losses.mean()
###################################################
# 하지만 logistic regression에 사용되는 cost function은 torch에서 제공을 해준다. : binary_cross_entropy함수.
cost=F.binary_cross_entropy(hypothesis,y_train)




In [2]:
# whole training

W=torch.zeros((2,1), requires_grad=True)
b=torch.zeros(1,requires_grad=True)

optimizer=torch.optim.SGD([W,b],lr=0.01)

nb_epochs=1000

for epoch in range(nb_epochs+1):
    
    hypothesis=torch.sigmoid(x_train.matmul(W)+b)
    cost=F.binary_cross_entropy(hypothesis,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.578086
Epoch  200/1000 Cost: 0.535207
Epoch  300/1000 Cost: 0.505647
Epoch  400/1000 Cost: 0.484024
Epoch  500/1000 Cost: 0.467288
Epoch  600/1000 Cost: 0.453674
Epoch  700/1000 Cost: 0.442127
Epoch  800/1000 Cost: 0.431998
Epoch  900/1000 Cost: 0.422874
Epoch 1000/1000 Cost: 0.414490


In [3]:
# Evaluation(내가 만든 모델의 정확도를 평가해보자.)
hypothesis=torch.sigmoid(x_train.matmul(W)+b)
print(hypothesis[:5])

tensor([[0.2169],
        [0.2644],
        [0.6868],
        [0.6295],
        [0.7870]], grad_fn=<SliceBackward0>)


In [4]:
prediction=hypothesis[:5]>=torch.FloatTensor([0.5])
print(prediction)
print(prediction.dtype)

tensor([[False],
        [False],
        [ True],
        [ True],
        [ True]])
torch.bool


In [5]:
correct_prediction=prediction.float()==y_train[:5]
print(correct_prediction)

tensor([[ True],
        [ True],
        [False],
        [ True],
        [ True]])


In [17]:
#Higher Implementation with Class

class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(2,1)           #참고로, torch.Linear는 함수가 아닌 모듈이다. 선형변환을 시켜주는 모듈을 함수처럼 호출한 것이다.
                                                #내부적으로 학습가능한 가중치와 편향을 가지고 있다. 
                                                #입력이 8개, 출력이 1개. 입력이 주어졌을 때, 그 입력에 대해 선형 변환을 할 수 있다.
        self.sigmoid=nn.Sigmoid()

    def forward(self,x):
        return self.sigmoid(self.linear(x))



In [18]:
# 전체적 코드
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)                #결과 재현을 위해 랜덤 시드를 정해주자.as_integer_ratio

x_data=[[1,2], [2,3], [3,1], [4,3], [5,3], [6,2]]
y_data=[[0],[0],[0],[1],[1],[1]]

x_train=torch.FloatTensor(x_data)
y_train=torch.FloatTensor(y_data)


model=BinaryClassifier()
optimizer=torch.optim.SGD(model.parameters(),lr=1)

nb_epochs=100
for epoch in range(nb_epochs+1):
    hypothesis=model(x_train)
    cost=F.binary_cross_entropy(hypothesis,y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch%10==0:
        prediction=(hypothesis>=torch.FloatTensor([0.5]))
        correct_prediction=prediction.float()==y_train
        accuracy=correct_prediction.sum().item()/len(correct_prediction)
        print('Epoch {:4d}/{} cost: {:.6f} Accuracy {:2.2f}%'.format(epoch, nb_epochs, cost.item(), accuracy*100,))

Epoch    0/100 cost: 0.539713 Accuracy 83.33%
Epoch   10/100 cost: 0.614853 Accuracy 66.67%
Epoch   20/100 cost: 0.441875 Accuracy 66.67%
Epoch   30/100 cost: 0.373145 Accuracy 83.33%
Epoch   40/100 cost: 0.316358 Accuracy 83.33%
Epoch   50/100 cost: 0.266094 Accuracy 83.33%
Epoch   60/100 cost: 0.220498 Accuracy 100.00%
Epoch   70/100 cost: 0.182095 Accuracy 100.00%
Epoch   80/100 cost: 0.157299 Accuracy 100.00%
Epoch   90/100 cost: 0.144091 Accuracy 100.00%
Epoch  100/100 cost: 0.134272 Accuracy 100.00%
